In [ ]:
# This file corresponses to the Figure 5. (How the combination of vertices and directions affect the classification results) 




In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 50
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru1_in = Reshape((6, 128*6))(pool4)
gru1 = GRU(32)(gru1_in)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(pool4)
gru2_in = Reshape((6, 128*6))(gru2_in_perm)
gru2 = GRU(32)(gru2_in)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2})(pool4)
gru3_in = Reshape((6, 128*6))(gru3_in_perm)
gru3 = GRU(32)(gru3_in)

gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru3_in_perm)
gru4_in = Reshape((6, 128*6))(gru4_in_perm)
gru4 = GRU(32)(gru4_in)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru5_in = Reshape((6, 128*6))(gru5_in_perm)
gru5 = GRU(32)(gru5_in)

gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru5_in_perm)
gru6_in = Reshape((6, 128*6))(gru6_in_perm)
gru6 = GRU(32)(gru6_in)

gru7_in_perm = Lambda(K.reverse,arguments={'axes':2})(gru5_in_perm)
gru7_in = Reshape((6, 128*6))(gru7_in_perm)
gru7 = GRU(32)(gru7_in)

gru8_in_perm = Lambda(K.reverse,arguments={'axes':1})(gru6_in_perm)
gru8_in = Reshape((6, 128*6))(gru8_in_perm)
gru8 = GRU(32)(gru8_in)

gru = Concatenate()([gru1,gru2,gru3,gru4,gru5,gru6,gru7,gru8])
gru_dropout = Dropout(0.5)(gru)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc_test = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc_test)
        print("AUCs:",auc_test)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn_losses(test).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru1_in = Reshape((6, 128*6))(pool4)
gru1 = GRU(32)(gru1_in)

gru_dropout = Dropout(0.1)(gru1)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc_test = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc_test)
        print("AUCs:",auc_test)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn1_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn1_losses(test).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output  
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(pool4)
gru2_in = Reshape((6, 128*6))(gru2_in_perm)
gru2 = GRU(32)(gru2_in)

gru_dropout = Dropout(0.1)(gru2)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc_test = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc_test)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn2_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn2_losses(test).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output  
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2})(pool4)
gru3_in = Reshape((6, 128*6))(gru3_in_perm)
gru3 = GRU(32)(gru3_in)

gru_dropout = Dropout(0.1)(gru3)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc_test = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc_test)
        print("AUCs:",auc_test)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn3_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn3_losses(test).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2})(pool4)
gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru3_in_perm)
gru4_in = Reshape((6, 128*6))(gru4_in_perm)
gru4 = GRU(32)(gru4_in)

gru_dropout = Dropout(0.1)(gru4)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc_test = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc_test)
        print("AUCs:",auc_test)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn4_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn4_losses(test).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru5_in = Reshape((6, 128*6))(gru5_in_perm)
gru5 = GRU(32)(gru5_in)

gru_dropout = Dropout(0.1)(gru5)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn5_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn5_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru5_in_perm)
gru6_in = Reshape((6, 128*6))(gru6_in_perm)
gru6 = GRU(32)(gru6_in)

gru_dropout = Dropout(0.1)(gru6)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn6_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn6_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru7_in_perm = Lambda(K.reverse,arguments={'axes':2})(gru5_in_perm)
gru7_in = Reshape((6, 128*6))(gru7_in_perm)
gru7 = GRU(32)(gru7_in)

gru_dropout = Dropout(0.1)(gru7)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn7_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn7_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru5_in_perm)
gru8_in_perm = Lambda(K.reverse,arguments={'axes':1})(gru6_in_perm)
gru8_in = Reshape((6, 128*6))(gru8_in_perm)
gru8 = GRU(32)(gru8_in)

gru_dropout = Dropout(0.1)(gru8)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn8_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn8_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 1360, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 1360, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 1360, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 340, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 340, 128)      73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 340, 128)      512       
__________

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru1_in = Reshape((6, 128*6))(pool4)
gru1 = GRU(32)(gru1_in)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(pool4)
gru2_in = Reshape((6, 128*6))(gru2_in_perm)
gru2 = GRU(32)(gru2_in)

gru = Concatenate()([gru1,gru2])
gru_dropout = Dropout(0.2)(gru)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn12_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn12_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2})(pool4)
gru3_in = Reshape((6, 128*6))(gru3_in_perm)
gru3 = GRU(32)(gru3_in)

gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru3_in_perm)
gru4_in = Reshape((6, 128*6))(gru4_in_perm)
gru4 = GRU(32)(gru4_in)

gru = Concatenate()([gru3,gru4])
gru_dropout = Dropout(0.2)(gru)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn34_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn34_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru5_in = Reshape((6, 128*6))(gru5_in_perm)
gru5 = GRU(32)(gru5_in)

gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru5_in_perm)
gru6_in = Reshape((6, 128*6))(gru6_in_perm)
gru6 = GRU(32)(gru6_in)

gru = Concatenate()([gru5,gru6])
gru_dropout = Dropout(0.2)(gru)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn56_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn56_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru5_in_perm)
gru7_in_perm = Lambda(K.reverse,arguments={'axes':2})(gru5_in_perm)
gru7_in = Reshape((6, 128*6))(gru7_in_perm)
gru7 = GRU(32)(gru7_in)

gru8_in_perm = Lambda(K.reverse,arguments={'axes':1})(gru6_in_perm)
gru8_in = Reshape((6, 128*6))(gru8_in_perm)
gru8 = GRU(32)(gru8_in)

gru = Concatenate()([gru7,gru8])
gru_dropout = Dropout(0.2)(gru)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn78_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn78_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru1_in = Reshape((6, 128*6))(pool4)
gru1 = GRU(32)(gru1_in)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2})(pool4)
gru3_in = Reshape((6, 128*6))(gru3_in_perm)
gru3 = GRU(32)(gru3_in)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru5_in = Reshape((6, 128*6))(gru5_in_perm)
gru5 = GRU(32)(gru5_in)

gru7_in_perm = Lambda(K.reverse,arguments={'axes':2})(gru5_in_perm)
gru7_in = Reshape((6, 128*6))(gru7_in_perm)
gru7 = GRU(32)(gru7_in)

gru = Concatenate()([gru1,gru3,gru5,gru7])
gru_dropout = Dropout(0.3)(gru)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn1357_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn1357_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [1]:
#  4 CNNs + GridRNN + output
import numpy as np
import keras.backend as K
from keras.callbacks import Callback
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Activation,Concatenate,Dense,Input,Dropout,Reshape,GRU,Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

epochs = 35
input_shape = (96,1360,1)
input_melS = Input(input_shape)

conv1 = Conv2D(64,(3,3),padding='same')(input_melS)
bn1 = BatchNormalization()(conv1)
act1 = Activation('elu')(bn1)
pool1 = MaxPooling2D((2,4),padding='same')(act1)

conv2 = Conv2D(128,(3,3),padding='same')(pool1)
bn2 = BatchNormalization()(conv2)
act2 = Activation('elu')(bn2)
pool2 = MaxPooling2D((2,4),padding='same')(act2)

conv3 = Conv2D(128,(3,3),padding='same')(pool2)
bn3 = BatchNormalization()(conv3)
act3 = Activation('elu')(bn3)
pool3 = MaxPooling2D((2,4),padding='same')(act3)

conv4 = Conv2D(128,(3,3),padding='same')(pool3)
bn4 = BatchNormalization()(conv4)
act4 = Activation('elu')(bn4)
pool4 = MaxPooling2D((2,4),padding='same')(act4)

gru2_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(pool4)
gru2_in = Reshape((6, 128*6))(gru2_in_perm)
gru2 = GRU(32)(gru2_in)

gru3_in_perm = Lambda(K.reverse,arguments={'axes':2})(pool4)
gru4_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru3_in_perm)
gru4_in = Reshape((6, 128*6))(gru4_in_perm)
gru4 = GRU(32)(gru4_in)

gru5_in_perm = Lambda(K.reverse,arguments={'axes':1})(pool4)
gru6_in_perm = Lambda(K.permute_dimensions,arguments={'pattern':(0,2,1,3)})(gru5_in_perm)
gru6_in = Reshape((6, 128*6))(gru6_in_perm)
gru6 = GRU(32)(gru6_in)

gru8_in_perm = Lambda(K.reverse,arguments={'axes':1})(gru6_in_perm)
gru8_in = Reshape((6, 128*6))(gru8_in_perm)
gru8 = GRU(32)(gru8_in)

gru = Concatenate()([gru2,gru4,gru6,gru8])
gru_dropout = Dropout(0.3)(gru)
fc = Dense(64, activation='elu')(gru_dropout)
pred = Dense(50, activation='sigmoid')(fc)

model = Model(inputs = [input_melS], outputs = [pred])
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = Adam())

path = 'C:/DT'
x_train = np.load(path+'\\x_train_MelSpec.npy')
y_train = np.load(path+'\\y_train.npy')
x_test = np.load(path+'\\x_test_MelSpec.npy')
y_test = np.load(path+'\\y_test.npy')

from sklearn.metrics import roc_auc_score
class Histories(Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(x_test)
        auc = roc_auc_score(y_test, y_pred)
        self.aucs.append(auc)
        print("AUCs:",auc)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

histories = Histories()
history = model.fit(x_train, y_train, epochs = epochs, callbacks=[histories])

import pickle
import os
save_path = 'C:/DT/Diff_directions'
pickle.dump(histories.aucs,open(os.path.join(save_path,'cGrnn2468_aucs(test).pkl'),'wb'))
pickle.dump(history.history,open(os.path.join(save_path,'cGrnn2468_losses(train).pkl'),'wb'))

C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
C:\Users\zwang10\AppData\Local\Continuum\Anaconda3\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 1360, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 1360, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1360, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 96, 1360, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [2]:
import os
import pickle

save_path = 'C:/DT/Diff_directions'
cGrnn_5 = pickle.load(open(os.path.join(save_path,'cGrnn1357_aucs(test).pkl'),'rb'))

max_auc = 0
idx = -1
for i in range(len(cGrnn_5)):
    if max_auc < cGrnn_5[i]:
        max_auc = cGrnn_5[i]
        idx = i
print(max_auc,idx+1)

0.8952426455986453 29
